To process your rules file, accounting for similarities like different versions of rules (e.g., `DQRC0001` and `DQRC0001_01`) or semantically similar rules (e.g., `DQRC0074` and `DQRC0081`), the code needs additional steps:

1. **Preprocessing and Similarity Analysis:**
   - Group rules with versioned IDs or highly similar logic.
   - Use a combination of string similarity (e.g., Levenshtein distance) and semantic similarity (e.g., cosine similarity on embeddings).
2. **Rule Clustering:**
   - Use text embeddings (via Sentence Transformers) to identify clusters of similar rules based on descriptions or code logic.

In [3]:
from pathlib import Path


path_root = Path.cwd()
while not (path_root / "data").exists():
    if path_root == path_root.parent:
        raise FileNotFoundError("Directory 'data' not found")
    path_root = path_root.parent

path_data = path_root / "data"

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import re

In [ ]:
pd.read_excel(path_root / 'data/processed/Bank_of_Rules-refractored.xlsx')

,RuleID,Code,Description,Category,Parameters
0,DQRA0194,<data>.apply(lambda x: x[cde] in <#>catalog<#>...,Ensures that the value in <cde> exists in a sp...,Validity,"['catalog', 'key', 'Column1', 'Lista']"
1,DQRAI0094,<data>[[cde]].apply(lambda x: True if (x[cde]....,Verifies that the value in <cde> starts with a...,Consistency,['startIndx']
2,DQRC0001,<data>.apply(lambda x: (not pd.isnull(x[cde]))...,Checks that the value in <cde> is not null and...,Completeness,[]
3,DQRC0001_01,<data>.apply(lambda x: (not pd.isnull(x[<#>Col...,Similar to DQRC0001 but applies the completene...,Completeness,"['Column1', 'Column1']"
4,DQRC0001_02,<data>.fillna({<#>bde_risk<#>: ' '}).apply(lam...,"Completeness check for <cde>, where missing va...",Completeness,['bde_risk']
...,...,...,...,...,...
143,DQRV0023,<data>[cde].isin(<#>Lista<#>),Checks if <cde> is part of a specified list (<...,Validity,['Lista']
144,DQRV0023_1,<data>[<#>Column1<#>].isin(<#>Lista<#>),Validates that <#>Column1<#> is part of a spec...,Validity,"['Column1', 'Lista']"
145,DQRV0023_2,~<data>[cde].isin(<#>Lista<#>),Ensures that <cde> is not part of a specified ...,Validity,['Lista']
146,DQRV0024,(( (<data>[cde] == <#>TipoPersona<#>) & (<data...,Ensures that if <cde> equals <#>TipoPersona<#>...,Other,"['TipoPersona', 'Column1', 'char']"


In [8]:
pd.read_json(path_root / 'data/internim/definitions.json')

,RuleID,Description,Condition/Logic,Parameters,Example,Category,Categorization
0,DQRC0001,Checks that the value in `<cde>` is not null a...,`<cde>` is non-null and contains non-whitespac...,{'<#>ColumnToCheck<#>': 'The column to validat...,"If `<cde>` = `Name` and `Name` = `John`, retur...",Completeness,Completeness / Non-Null Check
1,DQRA0194,Ensures that `<cde>` exists in `<#>catalog<#>`...,`<cde>` exists in `<#>catalog<#>` keyed by `<#...,{'<#>ColumnToCheck<#>': 'The column to validat...,If `<#>DependentColumn<#>`=`VIP` and `<#>ListA...,Catalog Check,Value List / Catalog Membership
2,DQRV0020,Compares `<cde>` with `<#>Column1<#>` as dates...,Parse `<cde>` using `<#>frmtDte1<#>` and `<#>C...,{'<#>ColumnToCheck1<#>': 'The first date colum...,If `<#>Operator<#>`=`==` and both dates parse ...,Date Check,Date Validation and Comparisons
3,DQRU0004,Ensures that `<cde>` does not contain duplicat...,`<cde>` must not have any duplicate values wit...,{'<#>ColumnToCheck<#>': 'The column to validat...,"If `<cde>`=`ID` and `ID`=[1, 2, 3, 3], return ...",Uniqueness Check,Uniqueness / Duplicate Checks
4,DQRF0005,Validates that the length of the value in `<cd...,The length of `<cde>` must satisfy `<#>Operato...,{'<#>ColumnToCheck<#>': 'The column to validat...,If `<cde>`=`Name` and `<#>Operator<#>`=`>` `<#...,Length Check,Data Type / Numeric / Length Constraints
5,DQRF0037,Applies a regex pattern (`<#>pattern<#>`) to v...,`<cde>` must match the provided `<#>pattern<#>...,{'<#>ColumnToCheck<#>': 'The column to validat...,If `<cde>`=`Code` and `<#>pattern<#>`=`^[A-Z]{...,Pattern Matching,Pattern Matching / Regex Validation
6,DQRI0106,Ensures that `<cde>` falls between `<#>Column2...,`<cde>` must satisfy `<#>Column2<#> <= <cde> <...,{'<#>ColumnToCheck<#>': 'The target column to ...,"If `<cde>`=`Score`=`50`, `<#>Column2<#>`=`40`,...",Range Check,Conditional Checks Based on Other Columns
7,DQRC0044,Checks if the value in `<cde>` is non-null and...,`<cde>` must be non-null and contain character...,{'<#>ColumnToCheck<#>': 'The column to validat...,If `<cde>`=`Address` and `Address`=`'123 Main ...,Completeness,Completeness / Non-Null Check
8,DQRF0006,Ensures that the length of `<cde>` falls betwe...,`<#>length_min<#>` `<#>Operator1<#>` length of...,{'<#>ColumnToCheck<#>': 'The column to validat...,"If `<cde>`=`Name`, `<#>length_min<#>`=`5`, `<#...",Length Check,Data Type / Numeric / Length Constraints
9,DQRF0178,Ensures that `<cde>` does not contain a decima...,The decimal part of `<cde>` must have at most ...,{'<#>ColumnToCheck<#>': 'The column to validat...,If `<cde>`=`Amount` and `<#>DecimalComplement<...,Numeric Validation,Data Type / Numeric / Length Constraints


In [10]:
# Load the dataset
rules_df = pd.read_excel(path_root / 'data/processed/Bank_of_Rules-refractored.xlsx')

# Step 1: Preprocess the data
def clean_text(text):
    # Remove placeholders and special characters for better text comparison
    return re.sub(r"<#>.*?<#>", "", text).strip()

rules_df['CleanedDescription'] = rules_df['Description'].apply(clean_text)

# Step 2: Generate semantic embeddings for rule descriptions
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(rules_df['CleanedDescription'].tolist())

# Step 3: Apply DBSCAN clustering on embeddings
dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

# Step 4: Assign clusters to the DataFrame
rules_df['Cluster'] = clusters

# Step 5: Identify versioned rules by RuleID patterns
def extract_base_rule_id(rule_id):
    return re.sub(r'_\d+$', '', rule_id)

rules_df['BaseRuleID'] = rules_df['RuleID'].apply(extract_base_rule_id)

# Group rules by BaseRuleID for analysis
rules_df['VersionGroup'] = rules_df.groupby('BaseRuleID').ngroup()

# Step 6: Save results for analysis
rules_df.to_csv(path_root / 'data/ClusteredAndGroupedRules.csv', index=False)

# Step 7: Optional - Analyze clusters
for cluster_id in sorted(rules_df['Cluster'].unique()):
    if cluster_id == -1:
        print("\nUnclustered rules:")
    else:
        print(f"\nCluster {cluster_id}:")
    print(rules_df[rules_df['Cluster'] == cluster_id][['RuleID', 'Description']])



Unclustered rules:
          RuleID                                        Description
113   DQRU0004_1  Removes duplicates from <#>Column1<#>, keeping...
132  DQRV0009_03  Ensures that <#>RecordID<#> (converted to floa...

Cluster 0:
          RuleID                                        Description
0       DQRA0194  Ensures that the value in <cde> exists in a sp...
1      DQRAI0094  Verifies that the value in <cde> starts with a...
2       DQRC0001  Checks that the value in <cde> is not null and...
3    DQRC0001_01  Similar to DQRC0001 but applies the completene...
4    DQRC0001_02  Completeness check for <cde>, where missing va...
..           ...                                                ...
143     DQRV0023  Checks if <cde> is part of a specified list (<...
144   DQRV0023_1  Validates that <#>Column1<#> is part of a spec...
145   DQRV0023_2  Ensures that <cde> is not part of a specified ...
146     DQRV0024  Ensures that if <cde> equals <#>TipoPersona<#>...
147     DQRV0032